### Gender Checker by Name

##### Introduction and Preprocessing

本节参考 [斗大的熊猫](http://blog.topspeedsnail.com/archives/10833) ，根据姓名判断性别

代码参考 [kaggle baby name competetion](https://github.com/tensorflow/models/tree/master/namignizer)

使用的数据集：[名字](https://pan.baidu.com/s/1hsHTEU4)

由于文件很大，无用字段很多，故此使用预处理脚本删除掉无用字段，同时还会生成全词汇数组文件 lexicon.pickle

> python2.7 sentiment140_preprocess.py

运行后得到 3 个结果文件

- data/sentiment140/lexicon.pickle  是由得到的 7176 个 lexicon 词典数组经过 cPickle.dump 得到的文件
- data/sentiment140/traindata   格式如 [1, 0, 0]:%:%:%[5217,2341,13]:%:%:%how are you，表示 positive，3个词在词典中位置为 5217,2341,13，不过顺序无关，也就是说 how 不一定在 5217 位置
- data/sentiment140/testdata   格式同上

上面的脚本需要使用 nltk 模块，且安装 nltk.download('punkt') 和 nltk.download('wordnet')

最后的 lexicon file 中共计 Total word count in lexicon: 7176 个词

然后还实现了一个 python module，叫做 sentiment140.py ，用于处理训练和测试数据，生成喂给 CNN 的训练和测试数据，主要实现两个方法

- get_train_batch(n=150)
- get_test_dataset()

这两个方法都返回 X, Y 两个数组

- X 数组中的每个元素也是一个数组，对应一条 tweet，长度为词典 lex 总数，该 tweet 在词典中存在的词的索引为 1， 否则为 0，即词 one-hot 编码之和
- Y 数组中的每个元素也是一个数组，对应一条 tweet 的标签，[1, 0, 0] 为 positive [0, 1, 0] 为 neutral [0, 0, 1] 为 negative

In [1]:
import tensorflow as tf
import numpy as np
import cPickle

from sentiment140 import Sentiment140

In [2]:
tf.__version__

'0.9.0'

##### Part I. 探索数据集

In [3]:
senti = Sentiment140()
test_x, test_y = senti.get_test_dataset()
print len(test_x), len(test_x[0])
print len(test_y), len(test_y[0])

498 7176
498 3


看到测试集中 498 个记录；每条记录的 x 为 7176 维，也即词典长度，y 为 3 维，也即 positive/neutral/negative

训练集过大，故此没有一次性导入，Sentiment140.py 提供一个取随机 batch 的函数；维度和上面是一致的

In [4]:
input_size = len(senti.lex)    # 这里就不去重新加载 lex 词典了，因为都已经在 Sentiment140 类中封装好了，故此通过这个方法取词典长度
num_classes = 3
print input_size

7176



##### Part II. Layer definations

In [5]:
# 定义参数


batch_size = 90    # 测试集 batch size

后面要做 embedding，故此这里先熟悉一下 tf.nn.embedding_lookup 函数
```
>>> sess = tf.InteractiveSession()
>>> w = tf.random_uniform([5, 2], -1.0, 1.0)       # 这里假设词典中词个数为 5 维，embedding 到 2 维，初始化为 -1~1 之间的随机数
>>> w = sess.run(w)
>>> x = [[1,1,0,0,0], [1,0,0,0,0], [0,0,1,1,1]]     # x 为 3 x 5 维， 3 代表 3 个样本，5 为词典中词个数，1 表示该词在词典中，0 表示不在
>>> embedded_chars = tf.nn.embedding_lookup(w, x)

>>> w
array([[ 0.82283998,  0.21245265],              # 看到，w 确实是随机出来了，每一行表示词典中的对应词的 2 维 embedding 向量
       [-0.737818  , -0.59785843],
       [-0.24692678, -0.69566345],
       [ 0.85945463, -0.21308041],
       [ 0.28053808,  0.88169646]], dtype=float32)
>>> x
[[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 1]]
>>> sess.run(embedded_chars)
array([[[-0.737818  , -0.59785843],             # 注意， embedding_chars 为 3 x 5 x 2 维，即样本个数 x 词典中词总个数 x embedding 维度
        [-0.737818  , -0.59785843],
        [ 0.82283998,  0.21245265],           # 我们以 embedding_chars[0::] 为例，也就是第一个样本得到的 lookup 结果
        [ 0.82283998,  0.21245265],           # 第一个样本为 [1,1,0,0,0]，即该样本中有词典的前 2 个词，而没有后面 3 个词
        [ 0.82283998,  0.21245265]],           # 我原以为 embedding_chars[0::] 会是前两个词的 embedding 向量 + 3 个 [0,0] 
                                    # 结果并不是这样，1 的部分填入 w[1]，而 0 的部分填入 w[0]
       [[-0.737818  , -0.59785843],           # 这样有一个问题，那就是 w[2:] 也就是 w 矩阵从第三个开始往后的元素都没有用到啊！
        [ 0.82283998,  0.21245265],           
        [ 0.82283998,  0.21245265],           # 保留疑问，后面再说
        [ 0.82283998,  0.21245265],
        [ 0.82283998,  0.21245265]],

       [[ 0.82283998,  0.21245265],
        [ 0.82283998,  0.21245265],
        [-0.737818  , -0.59785843],
        [-0.737818  , -0.59785843],
        [-0.737818  , -0.59785843]]], dtype=float32)
```

##### 更新认识  !!!

上面这个部分是之前做 Sentiment Analysis 时我的一些分析；而到现在我其实懂了，上面部分的 x 错了！

上面分析中认为，x 为一个矢量，维度和词典中词的个数相同，x 的每个元素表示词典对应位置的词是否存在，存在则 1 否则 0

实际上不是这样的，x 确实为一个矢量，只不过其长度和 x 这个句子的长度相同，每个元素则表示句子这个位置的词在词典中的索引，故此 x 的每个元素不限于 0 and 1，是句子中每个存在的词的索引

最后，为了 embedding 的需要，要求 batch 样本中所有句子对应的 x 有相同的长度，故此需要做一个 padding

这样：

- 传给 embedding 层的数据为 batch_size X padded_sentence_length，即 batch_size 个已经被 padding 为相同长度的句子 
- embedding 层的权重 W 为 vacabulary_size X embedding_size，表示词典中每个词对应的 embedding_size 长度的 vector
- embedding 层的结果就是 batch_size X padded_sentence_length X embedding_size，即 batch_size 个句子，每个句子的每个词由其在词典中的位置索引变为该词对应的 embedding 矢量

In [6]:
def neural_network(data, vocabulary_size, embedding_size = 128,  num_filters=128, dropout_keep_prob=0.5):
    """
    data 即 batch of X，维度为 sample_count x padded_sentence_length，后者记为 input_size
    """
    # embedding 层，把句子样本 batch 输入转为 vector
    with tf.name_scope("embedding"):
        # W 在 -1 & 1 之间随机分布，维度为 vocabulary_size x embedding_size
        W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        # 见上一个 cell 中的研究，embedded_char 为 sample_count x input_size x embedding_size 维
        embedded_chars = tf.nn.embedding_lookup(W, data)
        # 在最后再添加一个维度，那么就是 sample_count x input_size x embedding_size x 1 维
        # 我们知道 cnn 的输入为 sample_count x image_width x image_height x num_channel ，这里 num_channel 为 1
        # 然后，input_size x embedding_size 就相当于图片中的 width x height
        embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

    # 接下来是 CNN 层，注意和之前图片的串行 CNN 不同，这里采用的是并行 CNN，就是从 embedding 层出来的结果同时进入 3 个 CNN
    # 3 层 CNN 对应 3 个 filters，每个 filters 都是一维，或者理解为 n x 1 的二维； 每层 filter 都是 num_filter
    filter_sizes = [3, 4, 5]
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-{}-{}".format(i, filter_size)):
            # 准备初始化权重，首先得到权重的 shape
            # 输入的每个样本为 input_size x embedding_size ，那么我们把每层 filter 的维度设计为 filter_size x embedding_size
            # 也就是说，filter 的一个维度和样本相当，故此 filter 只会在 input_size 这个维度上滑动
            # 每层的 filter 都有 num_filters 组，而输入的 num_channel 都为 1
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]))
            # 每次移动一步, conv 为 sample_size x input_size x 1 x num_filters
            conv = tf.nn.conv2d(embedded_chars_expanded, W, strides=[1, 1, 1, 1], padding="VALID")
            h = tf.nn.relu(tf.nn.bias_add(conv, b))
            # pooling，pooling 跨度很大，input_size - filter_size + 1, 也就是说 input_size 那么长的向量，pool 完之后只剩 1 长
            # 故此 pooled 维度为 sample_size x 1 x 1 x num_filters
            pooled = tf.nn.max_pool(h, ksize=[1, input_size - filter_size + 1, 1, 1], strides=[1,1,1,1], padding="VALID")
            pooled_outputs.append(pooled)
    
    # pooled_output 为一个数组，每个元素为 pooled，pooled 的维度为 sample_size x 1 x 1 x num_filters
    num_filters_total = num_filters * len(filter_sizes)
    # 在 idx=3 的维度上 concat，结果维度为 sample_size x 1 x 1 x num_filters_total
    h_pool = tf.concat(3, pooled_outputs)     
    # 保持最后一个维度不变，进行 flaten，结果是二维的；第一个维度为 sample_size * 1 * 1，第二个维度仍为 num_filters_total
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])    
    
    # dropout
    with tf.name_scope("dropout"):
        # 仍然是 sample_size x num_filters_total 维度
        h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)
        
    # output full connection 层
    with tf.name_scope("output"):
        # W = tf.get_variable("W", shape=[num_filters_total, num_classes], initializer=tf.contrib.layers.xavier_initializer())
        W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.5))
        b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
        # sample_size x  num_classes 维度
        output = tf.nn.xw_plus_b(h_drop, W, b)
        return output

##### Part III. Trainning

In [7]:
X = tf.placeholder(tf.int32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, num_classes])
dropout_keep_prob = tf.placeholder(tf.float32)

In [10]:
output = neural_network(X, dropout_keep_prob)
optimizer = tf.train.AdamOptimizer(1e-3)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, Y))
# optimizer = optimizer.minimize(loss)
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars)   
    
y_pred = tf.argmax(tf.nn.softmax(output), 1)
correct_predictions = tf.equal(y_pred, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
    
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    i = 0
    pre_accuracy = 0
    no_improvement = 0
    while True:
        i += 1
        batch_x, batch_y = senti.get_train_batch(n=batch_size)
        sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y, dropout_keep_prob:0.5})
            
        if i % 10 == 0:
            accur = sess.run(accuracy, feed_dict={X:test_x[0:50], Y:test_y[0:50], dropout_keep_prob:1.0})
            print('accuracy:', accur)
            if accur > pre_accuracy:
                no_improvement = 0
                pre_accuracy = accur
                saver.save(sess, 'data/sentiment140/model.ckpt')
            else:
                no_improvement += 1
                if no_improvement == 5:
                    break


('accuracy:', 0.28)
('accuracy:', 0.51999998)
('accuracy:', 0.51999998)
('accuracy:', 0.30000001)
('accuracy:', 0.30000001)
('accuracy:', 0.30000001)
('accuracy:', 0.31999999)


看看如果是简单的双层 full connection layers 加一个 softmax 层如何

In [8]:
n_input_layer = len(senti.lex)
n_layer_1 = 2000
n_layer_2 = 2000
n_output_layer = 3  # 结果 3 维

def neural_network2(data):
    layer_1_w_b = {'w_':tf.Variable(tf.random_normal([n_input_layer, n_layer_1])), 'b_':tf.Variable(tf.random_normal([n_layer_1]))}
    layer_2_w_b = {'w_':tf.Variable(tf.random_normal([n_layer_1, n_layer_2])), 'b_':tf.Variable(tf.random_normal([n_layer_2]))}
    layer_output_w_b = {'w_':tf.Variable(tf.random_normal([n_layer_2, n_output_layer])), 'b_':tf.Variable(tf.random_normal([n_output_layer]))}
 
    layer_1 = tf.add(tf.matmul(tf.cast(data, tf.float32), layer_1_w_b['w_']), layer_1_w_b['b_'])
    # layer_1 = tf.nn.xw_plus_b(tf.cast(data, tf.float32), layer_1_w_b['w_'], layer_1_w_b['b_'])   # 同上面
    layer_1 = tf.nn.relu(layer_1)  # 激活函数
    # layer_2 = tf.add(tf.matmul(layer_1, layer_2_w_b['w_']), layer_2_w_b['b_']) # 同下面
    layer_2 = tf.nn.xw_plus_b(layer_1, layer_2_w_b['w_'], layer_2_w_b['b_'])
    layer_2 = tf.nn.relu(layer_2 ) # 激活函数
    layer_output = tf.add(tf.matmul(layer_2, layer_output_w_b['w_']), layer_output_w_b['b_'])
 
    return layer_output

In [11]:
predict = neural_network2(X)
cost_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(predict, Y))
optimizer = tf.train.AdamOptimizer().minimize(cost_func)
correct = tf.equal(tf.argmax(predict,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct,'float'))

with tf.Session() as session:
    session.run(tf.initialize_all_variables())
 
    saver = tf.train.Saver()
    i = 0
    no_improvement = 0
    pre_accuracy = 0
    while True:   # 一直训练
        i += 1
        batch_x, batch_y = senti.get_train_batch(n=batch_size)
        _, trainaccur = session.run([optimizer, accuracy], feed_dict={X:batch_x,Y:batch_y})
 
        if i % 10 == 0:
            print ('training accuracy', trainaccur)
            accur = session.run(accuracy, feed_dict={X:test_x[50:100], Y:test_y[50:100]})
            print('testing accuracy:', accur)
            if accur > pre_accuracy:
                no_improvement = 0
                pre_accuracy = accur
                # saver.save(session, 'data/sentiment140/model.ckpt')
            else:
                no_improvement += 1
                if no_improvement == 20:
                    break

('training accuracy', 0.53333336)
('testing accuracy:', 0.47999999)
('training accuracy', 0.5777778)
('testing accuracy:', 0.44)
('training accuracy', 0.51111114)
('testing accuracy:', 0.44)
('training accuracy', 0.63333333)
('testing accuracy:', 0.47999999)
('training accuracy', 0.63333333)
('testing accuracy:', 0.46000001)
('training accuracy', 0.58888888)
('testing accuracy:', 0.5)
('training accuracy', 0.60000002)
('testing accuracy:', 0.54000002)
('training accuracy', 0.56666666)
('testing accuracy:', 0.54000002)
('training accuracy', 0.60000002)
('testing accuracy:', 0.47999999)
('training accuracy', 0.60000002)
('testing accuracy:', 0.54000002)
('training accuracy', 0.51111114)
('testing accuracy:', 0.54000002)
('training accuracy', 0.65555555)
('testing accuracy:', 0.51999998)
('training accuracy', 0.55555558)
('testing accuracy:', 0.51999998)
('training accuracy', 0.63333333)
('testing accuracy:', 0.54000002)
('training accuracy', 0.64444447)
('testing accuracy:', 0.51999998)


KeyboardInterrupt: 

看到，无论是简单的神经网络还是 CNN 结果都并不理想